In [1]:
import time
import random
import datetime
import json
from arduino_iot_cloud import ArduinoCloudClient

# Arduino Cloud credentials
DEVICE_ID = "0291f60a-cfaf-462d-9e82-5ce662fb3823"
SECRET_KEY = "llXRFeQyYtfm!EXXIIvXygwmD"
THING_ID = "e8ea4656-01da-4a15-9eb5-3e8053a043f8"

# Arduino Cloud credentials from your device setup
# DEVICE_ID and SECRET_KEY are provided when you create a device in Arduino Cloud

WEBHOOK_SECRET = "mySuperSecret123"

# Initial battery state with all required fields
battery_state = {
    "id": 1,
    "battery_id": 1,
    "state_of_charge": 80.0,
    "voltage": 11.5,
    "current_amps": 1.5,
    "power_watts": 17.25,
    "time_remaining": 240,
    "temp_battery": 25.0,
    "amp_hours_consumed": 0.0,
    "charging_current": 0.0,
    "timestamp": int(time.time()),
    "usb_voltage": 5.0,
    "usb_power": 2.5,
    "usb_current": 0.5,
    "latitude": 37.7749,
    "longitude": -122.4194,
    "altitude": 15.0,
    "SD_card_storage_remaining": 1024.0,
    "battery_orientation": "normal",
    "number_GPS_satellites_for_fix": 8,
    "mobile_signal_strength": 3,
    "event_type": "normal_operation",
    "new_battery_cycle": 0
}

def clamp(val, min_val, max_val):
    return max(min_val, min(max_val, val))

def update_battery_state(state):
    # Update timestamp
    state["timestamp"] = int(time.time())
    
    # Simulate state of charge change (decrease if not charging, increase if charging)
    is_charging = random.random() < 0.3  # 30% chance of charging
    
    if is_charging:
        charge_change = random.uniform(0.1, 0.5)
        state["charging_current"] = random.uniform(1.0, 3.0)
    else:
        charge_change = random.uniform(-0.5, -0.1)
        state["charging_current"] = 0.0
    
    state["state_of_charge"] = clamp(state["state_of_charge"] + charge_change, 0, 100)
    
    # Update voltage based on state of charge
    base_voltage = 8 + (state["state_of_charge"] / 100) * 4
    state["voltage"] = clamp(base_voltage + random.uniform(-0.1, 0.1), 8, 12)
    
    # Update current draw
    state["current_amps"] = random.uniform(0, 5)
    
    # Calculate power
    state["power_watts"] = state["voltage"] * state["current_amps"]
    
    # Update time remaining (based on charge level and current draw)
    if state["current_amps"] > 0:
        # Assuming a 50Ah battery
        capacity_remaining = 50 * (state["state_of_charge"] / 100)
        hours_remaining = capacity_remaining / state["current_amps"]
        state["time_remaining"] = int(hours_remaining * 60)  # Convert to minutes
    else:
        state["time_remaining"] = 0
    
    # Update amp hours consumed (incremental)
    time_fraction = 10 / 3600  # 10 seconds in hours
    state["amp_hours_consumed"] += state["current_amps"] * time_fraction
    
    # Update temperature (random fluctuation)
    state["temp_battery"] = clamp(state["temp_battery"] + random.uniform(-0.5, 0.5), 15, 45)
    
    # Update USB-related metrics
    state["usb_voltage"] = clamp(5.0 + random.uniform(-0.2, 0.2), 4.8, 5.2)
    state["usb_current"] = clamp(state["usb_current"] + random.uniform(-0.1, 0.1), 0, 2.0)
    state["usb_power"] = state["usb_voltage"] * state["usb_current"]
    
    # Simulate slight GPS drift
    state["latitude"] += random.uniform(-0.0001, 0.0001)
    state["longitude"] += random.uniform(-0.0001, 0.0001)
    state["altitude"] += random.uniform(-0.1, 0.1)
    
    # Update storage
    state["SD_card_storage_remaining"] -= random.uniform(0, 0.5)  # Lose up to 0.5 MB
    
    # Randomly change orientation sometimes
    if random.random() < 0.05:  # 5% chance
        orientations = ["normal", "upside_down", "sideways", "tilted"]
        state["battery_orientation"] = random.choice(orientations)
    
    # Update GPS satellites (fluctuates)
    state["number_GPS_satellites_for_fix"] = clamp(
        state["number_GPS_satellites_for_fix"] + random.choice([-1, 0, 0, 0, 1]), 
        4, 12
    )
    
    # Update mobile signal (fluctuates)
    state["mobile_signal_strength"] = clamp(
        state["mobile_signal_strength"] + random.choice([-1, 0, 0, 0, 1]), 
        0, 5
    )
    
    # Occasionally change event type
    if random.random() < 0.05:  # 5% chance
        events = ["normal_operation", "low_battery", "charging", "discharging", "error", "warning"]
        state["event_type"] = random.choice(events)
    
    # Very rarely start a new battery cycle
    if random.random() < 0.01:  # 1% chance
        state["new_battery_cycle"] = 1
    else:
        state["new_battery_cycle"] = 0
    
    return state

def send_to_arduino_cloud(client, state):
    try:
        # Format timestamp for better readability
        state_copy = state.copy()
        timestamp_unix = state_copy["timestamp"]
        state_copy["timestamp"] = datetime.datetime.fromtimestamp(timestamp_unix).isoformat()
        
        # Create a wrapper structure that separates auth from data
        wrapper = {
            "auth": {
                "secret": WEBHOOK_SECRET
            },
            "data": state_copy
        }
        
        # Convert the wrapper to a JSON string
        json_value = json.dumps(wrapper)
        
        # Register the property with the client before sending data
        # You need to register properties before you can update them
        if not hasattr(client, '_properties_registered'):
            client.register('data')  # Register your property name
            client._properties_registered = True
        
        # Update the property using dictionary-style access
        client['data'] = json_value
        
        print(f"Data successfully sent to Arduino Cloud")
        
        # Print a summary of the data that was sent
        print(f"SoC: {state['state_of_charge']:.1f}%, Voltage: {state['voltage']:.2f}V, " +
              f"Current: {state['current_amps']:.2f}A, Power: {state['power_watts']:.2f}W")
    except Exception as e:
        print(f"Error sending data: {e}")


In [4]:
client = ArduinoCloudClient(
    device_id=DEVICE_ID, 
    username=DEVICE_ID, 
    password=SECRET_KEY,
    sync_mode=True
)
client.start()
client.register("data", value="{'testing':123}")
client.update()

In [9]:
client.start()

In [10]:
client.register("data", value="{'testing':123}")

In [11]:
client.update()